# LLama doctor

This project would like to examine the architecture of current transformers to improve their efficiency for possible deployment on a micro-controller unit. Currently, some research are done on Diffusion model that tries to reduce the number of iterations applied on input noise to obtain a final image.

In [ ]:
import numpy